In [85]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS
from dotenv import load_dotenv
from urllib.parse import urlparse, unquote

In [86]:
# Load the environment variables from the .env file
load_dotenv()

xml_folder_path = os.getenv('TEST_FILES')
tsv_file = os.getenv('FILE_TSV')

# Leggi il file TSV con le informazioni sui medicinali
with open(tsv_file, 'r') as tsv_file:
    df = pd.read_csv(tsv_file, delimiter='\t')

# Creazione del grafo RDF in formato JSON-LD
graph = Graph()

# Definizione di namespace RDF e altri namespace personalizzati
CT = Namespace("https://clinicaltrials.gov/")
DRUG = Namespace("https://example.org/drug#")

graph.bind('ct', CT)
graph.bind('drug', DRUG)

# Dizionario per memorizzare le informazioni associate a ciascun farmaco
drugs_info = {}

# Dizionario per tenere traccia dei file in cui appare ciascun farmaco
drug_files = {}

# Itera su tutti i file XML nella cartella
for filename in os.listdir(xml_folder_path):
    if filename.endswith('.xml'):
        xml_file_path = os.path.join(xml_folder_path, filename)

        tree = ET.parse(xml_file_path)
        root = tree.getroot()

        # Estrai informazioni dal file XML
        intervention_type = root.find('.//intervention_type')
        intervention_name = root.find('.//intervention_name')

        # Verifica se il campo intervention_type esiste prima di procedere
        if intervention_type is not None and intervention_type.text.lower() == 'drug':
            # Cerca corrispondenze nel file TSV per il nome del farmaco (case-insensitive)
            drug_info = df[df['DRUG_NAME'].str.lower() == intervention_name.text.lower()]

            for index, row in drug_info.iterrows():
                drug_name = row['DRUG_NAME']
                moa_source_url = row['MOA_SOURCE_URL']
                act_source_url = row['ACT_SOURCE_URL']

                # Sostituisci spazi con underscore per creare URI validi
                drug_name_uri = '_'.join(drug_name.split())

                # Inizializza il dizionario per il farmaco se non esiste
                if drug_name_uri not in drugs_info:
                    drugs_info[drug_name_uri] = {
                        'moa_urls': [],
                        'act_urls': [],
                    }

                # Verifica se il valore è NaN prima di normalizzare l'URL
                if not pd.isna(moa_source_url):
                    # Normalizza gli URL
                    moa_source_url = unquote(str(moa_source_url))
                    drugs_info[drug_name_uri]['moa_urls'].append(Literal(moa_source_url))

                # Verifica se il valore è NaN prima di normalizzare l'URL
                if not pd.isna(act_source_url):
                    # Normalizza gli URL
                    act_source_url = unquote(str(act_source_url))
                    drugs_info[drug_name_uri]['act_urls'].append(Literal(act_source_url))

                # Aggiungi il file corrente all'elenco dei file per questo farmaco
                if drug_name_uri not in drug_files:
                    drug_files[drug_name_uri] = []
                drug_files[drug_name_uri].append(filename)

# Aggiungi informazioni al grafo RDF
for drug_name_uri, info in drugs_info.items():
    drug_uri = URIRef(DRUG[drug_name_uri])
    graph.add((drug_uri, RDF.type, DRUG.Drug))
    graph.add((drug_uri, RDFS.label, Literal(drug_name_uri.replace('_', ' '))))

    # Aggiungi array MOA source URL se presente, altrimenti aggiungi "None"
    for moa_url in info['moa_urls']:
        graph.add((drug_uri, DRUG.moaSourceURL, moa_url))
    if not info['moa_urls']:
        graph.add((drug_uri, DRUG.moaSourceURL, Literal(None)))

    # Aggiungi array ACT source URL se presente, altrimenti aggiungi "None"
    for act_url in info['act_urls']:
        graph.add((drug_uri, DRUG.actSourceURL, act_url))
    if not info['act_urls']:
        graph.add((drug_uri, DRUG.actSourceURL, Literal(None)))

# Verifica se più di un file fa riferimento allo stesso farmaco
for drug_name_uri, files in drug_files.items():
    if len(files) > 1:
        print(f"Il farmaco {drug_name_uri.replace('_', ' ')} è menzionato nei seguenti file: {', '.join(files)}")

# Serializza il grafo RDF in formato JSON-LD
jsonld_data = graph.serialize(format='json-ld', indent=4)

print(jsonld_data)

# Salva il risultato su un file JSON-LD
# with open('output.jsonld', 'w') as f:
#     f.write(jsonld_data)


Il farmaco fluorouracil è menzionato nei seguenti file: NCT00010088.xml, NCT00010088.xml, NCT00010088.xml, NCT00010088.xml, NCT00010088.xml, NCT00010075.xml, NCT00010075.xml, NCT00010075.xml, NCT00010075.xml, NCT00010075.xml, NCT00010062.xml, NCT00010062.xml, NCT00010062.xml, NCT00010062.xml, NCT00010062.xml
Il farmaco fulvestrant è menzionato nei seguenti file: NCT00010153.xml, NCT00010153.xml, NCT00010153.xml, NCT00010153.xml, NCT00010153.xml, NCT00010153.xml, NCT00010153.xml, NCT00010153.xml, NCT00010153.xml, NCT00010153.xml, NCT00010153.xml
Il farmaco exemestane è menzionato nei seguenti file: NCT00010010.xml, NCT00010010.xml
Il farmaco docetaxel è menzionato nei seguenti file: NCT00010140.xml, NCT00010140.xml, NCT00010140.xml, NCT00010140.xml, NCT00010140.xml, NCT00010140.xml, NCT00010140.xml, NCT00010140.xml
[
    {
        "@id": "https://example.org/drug#exemestane",
        "@type": [
            "https://example.org/drug#Drug"
        ],
        "http://www.w3.org/2000/01/rdf

In [87]:
# Load the environment variables from the .env file
load_dotenv()

xml_folder_path = os.getenv('TEST_FILES')
tsv_file = os.getenv('FILE_TSV')

# Creazione del grafo RDF in formato JSON-LD
graph = Graph()

# Definizione di namespace RDF e altri namespace personalizzati
CT = Namespace("https://clinicaltrials.gov/")
DRUG = Namespace("https://example.org/drug#")
FILE = Namespace("https://example.org/file#")

graph.bind('ct', CT)
graph.bind('drug', DRUG)
graph.bind('file', FILE)

# Dizionario per memorizzare le informazioni associate a ciascun farmaco
drugs_info = {}

# Dizionario per tenere traccia dei file in cui appare ciascun farmaco
drug_files = {}

# Insieme per tenere traccia dei farmaci menzionati nei file XML
mentioned_drugs = set()

# Itera su tutti i file XML nella cartella
for filename in os.listdir(xml_folder_path):
    if filename.endswith('.xml'):
        xml_file_path = os.path.join(xml_folder_path, filename)

        tree = ET.parse(xml_file_path)
        root = tree.getroot()

        # Estrai informazioni dal file XML
        intervention_type = root.find('.//intervention_type')
        intervention_name = root.find('.//intervention_name')

        # Verifica se il campo intervention_type esiste prima di procedere
        if intervention_type is not None and intervention_type.text.lower() == 'drug':
            # Aggiungi il nome del farmaco all'insieme dei farmaci menzionati
            mentioned_drugs.add(intervention_name.text.lower())

# Leggi il file TSV con le informazioni sui medicinali
with open(tsv_file, 'r') as tsv_file:
    df = pd.read_csv(tsv_file, delimiter='\t')

# Filtra il DataFrame per includere solo i farmaci menzionati nei file XML
df = df[df['DRUG_NAME'].str.lower().isin(mentioned_drugs)]

# Itera sui farmaci menzionati nei file XML
for index, row in df.iterrows():
    drug_name = row['DRUG_NAME']
    moa_source_url = row['MOA_SOURCE_URL']
    act_source_url = row['ACT_SOURCE_URL']

    # Sostituisci spazi con underscore per creare URI validi
    drug_name_uri = '_'.join(drug_name.split())

    # Inizializza il dizionario per il farmaco se non esiste
    if drug_name_uri not in drugs_info:
        drugs_info[drug_name_uri] = {
            'moa_urls': [],
            'act_urls': [],
        }

    # Verifica se il valore è NaN prima di normalizzare l'URL
    if not pd.isna(moa_source_url):
        # Normalizza gli URL
        moa_source_url = unquote(str(moa_source_url))
        drugs_info[drug_name_uri]['moa_urls'].append(Literal(moa_source_url))

    # Verifica se il valore è NaN prima di normalizzare l'URL
    if not pd.isna(act_source_url):
        # Normalizza gli URL
        act_source_url = unquote(str(act_source_url))
        drugs_info[drug_name_uri]['act_urls'].append(Literal(act_source_url))

    # Collega il file corrente al nodo del farmaco
    if drug_name_uri not in drug_files:
        drug_files[drug_name_uri] = []
    drug_files[drug_name_uri].append(filename)

# Aggiungi informazioni al grafo RDF
for drug_name_uri, info in drugs_info.items():
    drug_uri = URIRef(DRUG[drug_name_uri])
    graph.add((drug_uri, RDF.type, DRUG.Drug))
    graph.add((drug_uri, RDFS.label, Literal(drug_name_uri.replace('_', ' '))))

    # Aggiungi array MOA source URL se presente, altrimenti aggiungi "None"
    for moa_url in info['moa_urls']:
        graph.add((drug_uri, DRUG.moaSourceURL, moa_url))
    if not info['moa_urls']:
        graph.add((drug_uri, DRUG.moaSourceURL, Literal(None)))

    # Aggiungi array ACT source URL se presente, altrimenti aggiungi "None"
    for act_url in info['act_urls']:
        graph.add((drug_uri, DRUG.actSourceURL, act_url))
    if not info['act_urls']:
        graph.add((drug_uri, DRUG.actSourceURL, Literal(None)))

    # Collega il file corrente al nodo del farmaco
    for filename in drug_files[drug_name_uri]:
        file_uri = URIRef(FILE[filename])
        graph.add((file_uri, RDF.type, FILE.File))
        graph.add((file_uri, FILE.references, drug_uri))

# Serializza il grafo RDF in formato JSON-LD
jsonld_data = graph.serialize(format='json-ld', indent=4)

print(jsonld_data)

# Salva il risultato su un file JSON-LD
# with open('output.jsonld', 'w') as f:
#     f.write(jsonld_data)


[
    {
        "@id": "https://example.org/drug#exemestane",
        "@type": [
            "https://example.org/drug#Drug"
        ],
        "http://www.w3.org/2000/01/rdf-schema#label": [
            {
                "@value": "exemestane"
            }
        ],
        "https://example.org/drug#actSourceURL": [
            {
                "@value": "None"
            }
        ],
        "https://example.org/drug#moaSourceURL": [
            {
                "@value": "https://www.ebi.ac.uk/chembl/compound/inspect/CHEMBL1200374"
            }
        ]
    },
    {
        "@id": "https://example.org/drug#fluorouracil",
        "@type": [
            "https://example.org/drug#Drug"
        ],
        "http://www.w3.org/2000/01/rdf-schema#label": [
            {
                "@value": "fluorouracil"
            }
        ],
        "https://example.org/drug#actSourceURL": [
            {
                "@value": "None"
            }
        ],
        "https://example.org